In [ ]:
import torch
import gradio as gr
from transformers import pipeline

# 1. Hugging Face 모델 설정
# 음성을 텍스트로 변환하기 위한 Whisper 모델과 텍스트 요약을 위한 Hugging Face LLM을 설정합니다.
speech_to_text_model = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-tiny.en",
    chunk_length_s=30,
)

text_summarization_model = pipeline(
    "text2text-generation",
    model="google/flan-t5-small",  # 무료로 사용할 수 있는 작은 모델
    max_length=100,
    temperature=0.7,
)

# 2. 텍스트 요약을 위한 프롬프트 템플릿
def generate_summary(context): # 질문 : context는 어디서 오는값이야?
    prompt = f"Summarize the following text:\n{context}" # 질문 : 이거를 보면 이것도 딕셔너리 같은데 왜 이렇게 쓰인거야? 
    summary = text_summarization_model(prompt)[0]["generated_text"] # 질문 : summarizationmodel에 dictionary 형태의 prompt를 넣었을때 생긴 새로운 딕셔너리의 키값이 generated text인거고 첫번째 value 불러오는거지? 
    return summary

# 3. 음성 -> 텍스트 -> 요약 처리 함수
def transcript_and_summarize(audio_file):
    # 음성을 텍스트로 변환
    transcript_txt = speech_to_text_model(audio_file, batch_size=8)["text"] # 질문 : 이거 batchsize 8은 뭐야? 일단 audio file을 model에 넣고 text를 키로 가진 딕셔너리를 만드는거지? 

    # 변환된 텍스트를 요약
    summary = generate_summary(transcript_txt) # 질문 : 아까 context로 이미 generated text 했는데 또 transcript 넣어? 

    return {
        "Transcription": transcript_txt, # 질문 : 이거 결과가 transcription : 원문 오디오 텍스트 이렇게 나와? 
        "Summary": summary,
    }

# 4. Gradio 인터페이스 설정
audio_input = gr.Audio(sources="upload", type="filepath", label="Upload Audio File")  # 오디오 파일 업로드
transcription_output = gr.Textbox(label="Transcription")
summary_output = gr.Textbox(label="Summary")

iface = gr.Interface(
    fn=transcript_and_summarize,  # 함수 연결
    inputs=audio_input,
    outputs=[transcription_output, summary_output],
    title="Audio Transcription and Summarization App",
    description="Upload an audio file to transcribe and summarize its content."
)

# 5. Gradio 앱 실행
if __name__ == "__main__":
    iface.launch(server_name="0.0.0.0", server_port=7860)
